In [1]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path

# File to Load (Remember to Change These)
school_data_to_load = Path("Resources/schools_complete.csv")
student_data_to_load = Path("Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head(10000)

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635
...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,Patrick Campbell,M,12th,Hernandez High School,65,83,3,District,4635,3022020
9996,9996,Kristen Barrera,F,11th,Hernandez High School,69,73,3,District,4635,3022020
9997,9997,Eric Hill,M,11th,Hernandez High School,72,85,3,District,4635,3022020
9998,9998,Steven Fox,M,9th,Hernandez High School,85,67,3,District,4635,3022020


In [2]:
school_data_complete["type"].unique()

array(['District', 'Charter'], dtype=object)

In [3]:
# Total number of unique schools
school_count = school_data_complete["school_name"].nunique()
school_count

15

In [4]:
# Total Students
student_count = school_data_complete["student_name"].count()
student_count

39170

In [5]:
# Since the number of budgets is the same as the number of unique schools we can sum the unique values. 
# We wouldn't be able to do this if some schools had the same budget
school_data_complete["budget"].nunique() == school_data_complete["school_name"].nunique()

True

In [6]:
# Total budget
total_budget = school_data_complete["budget"].unique().sum()
total_budget

24649428

In [7]:
# Average math score
average_math_score = school_data_complete["math_score"].mean()
average_math_score

78.98537145774827

In [8]:
# Average reading score
average_reading_score = school_data_complete["reading_score"].mean()
average_reading_score

81.87784018381414

In [9]:
# Percentage of students who passed math
passing_math_students = ((school_data_complete[school_data_complete["math_score"] >= 70].count()["student_name"]) /float(school_data_complete["student_name"].count())) * 100
passing_math_students

74.9808526933878

In [10]:
# Percentage of students who passed reading
passing_reading_students = ((school_data_complete[school_data_complete["reading_score"] >= 70].count()["student_name"])/school_data_complete["student_name"].count()) * 100
passing_reading_students


passing_reading_students = ((school_data_complete[school_data_complete["reading_score"] >= 70].count()["student_name"]) /float(school_data_complete["student_name"].count())) * 100
passing_reading_students

85.80546336482001

In [11]:
# Percentage of students who passed both math and reading
passing_math_reading_count = school_data_complete[
    (school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)
].count()["student_name"]

overall_passing_rate = passing_math_reading_count /  float(student_count) * 100
overall_passing_rate

65.17232575950983

In [12]:
# Create a high-level snapshot of the district's key metrics in a DataFrame
district_summary = pd.DataFrame({
    "Total Schools": [school_count],
    "Total Students": [student_count],
    "Total Budget": [total_budget],
    "Average Math Score": [average_math_score],
    "Average Reading Score": [average_reading_score],
    "% Passing Math": [passing_math_students],
    "% Passing Reading": [passing_reading_students],
    "% Overall Passing": [overall_passing_rate],
})

# Formatting
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

# Format columns
for column in district_summary[["% Passing Math", "% Passing Reading", "% Overall Passing"]]:
    district_summary[column] = district_summary[column].map("{:,.2f}%".format)

# Display the DataFrame
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.98%,85.81%,65.17%


In [13]:
# Function that will perform necessary analysis and add to a dataframe.
# Input is a dataframe with school data and the summary dataframe to add the stats to

def school_stats(school_df, summary_df):
    
    school_name = school_df["school_name"].unique()[0]

    school_type = school_df["type"].unique()[0]
    
    student_count = school_df["student_name"].count()
    
    total_budget = school_df["budget"].unique()[0]
    
    per_student_budget = total_budget/student_count
    
    average_math_score = school_df["math_score"].mean()
    
    average_reading_score = school_df["reading_score"].mean()
    
    passing_math_students = ((school_df[school_df["math_score"] >= 70].count()["student_name"]) /float(school_df["student_name"].count())) * 100
    
    passing_reading_students = ((school_df[school_df["reading_score"] >= 70].count()["student_name"]) /float(school_df["student_name"].count())) * 100
    
    passing_math_reading_count = school_df[
    (school_df["math_score"] >= 70) & (school_df["reading_score"] >= 70)].count()["student_name"]

    overall_passing_rate = passing_math_reading_count /  float(student_count) * 100 
    
    summary_row = {"School Name": school_name, "School Type": school_type, "Total Students": student_count, "Total Budget": total_budget,\
                  "Per Student Budget": per_student_budget, "Average Math Score": average_math_score, "Average Reading Score": average_reading_score,\
                  "% Passing Math": passing_math_students, "% Passing Reading": passing_reading_students, "% Overall Passing": overall_passing_rate}
        
    summary_df = pd.concat( [ summary_df, pd.DataFrame.from_records([summary_row]) ], ignore_index=True )
    
    return summary_df


In [14]:
# Initialize summary dataframe
per_school_summary = pd.DataFrame({
    "School Name": [],
    "School Type": [],
    "Total Students": [],
    "Total Budget": [],
    "Per Student Budget": [],
    "Average Math Score": [],
    "Average Reading Score": [],
    "% Passing Math": [],
    "% Passing Reading": [],
    "% Overall Passing": [],
})

In [15]:
# Analyze each school and add to summary dataframe
for school in school_data_complete["school_name"].unique():
    per_school_summary = school_stats(school_data_complete[school_data_complete["school_name"] == school], per_school_summary)

In [16]:
# Reset index to School Name
per_school_summary.set_index("School Name", drop=True)

,School Type,Total Students,Total Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Huang High School,District,2917.0,1910635.0,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
Figueroa High School,District,2949.0,1884411.0,639.0,76.711767,81.158020,65.988471,80.739234,53.204476
Shelton High School,Charter,1761.0,1056600.0,600.0,83.359455,83.725724,93.867121,95.854628,89.892107
Hernandez High School,District,4635.0,3022020.0,652.0,77.289752,80.934412,66.752967,80.862999,53.527508
Griffin High School,Charter,1468.0,917500.0,625.0,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283.0,1319574.0,578.0,83.274201,83.989488,93.867718,96.539641,90.582567
Cabrera High School,Charter,1858.0,1081356.0,582.0,83.061895,83.975780,94.133477,97.039828,91.334769
Bailey High School,District,4976.0,3124928.0,628.0,77.048432,81.033963,66.680064,81.933280,54.642283
Holden High School,Charter,427.0,248087.0,581.0,83.803279,83.814988,92.505855,96.252927,89.227166


In [17]:
# Highest-Performing Schools (by % Overall Passing)
top_schools = per_school_summary.sort_values(by=['% Overall Passing'], ascending=False).head(5)
top_schools.reset_index(drop="True")

,School Name,School Type,Total Students,Total Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,Cabrera High School,Charter,1858.0,1081356.0,582.0,83.061895,83.975780,94.133477,97.039828,91.334769
1,Thomas High School,Charter,1635.0,1043130.0,638.0,83.418349,83.848930,93.272171,97.308869,90.948012
2,Griffin High School,Charter,1468.0,917500.0,625.0,83.351499,83.816757,93.392371,97.138965,90.599455
3,Wilson High School,Charter,2283.0,1319574.0,578.0,83.274201,83.989488,93.867718,96.539641,90.582567
4,Pena High School,Charter,962.0,585858.0,609.0,83.839917,84.044699,94.594595,95.945946,90.540541


In [18]:
# Lowest-Performing Schools (by % Overall Passing)
bottom_schools = per_school_summary.sort_values(by=['% Overall Passing'], ascending=True).head(5)
bottom_schools.reset_index(drop="True")

,School Name,School Type,Total Students,Total Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,Rodriguez High School,District,3999.0,2547363.0,637.0,76.842711,80.744686,66.366592,80.220055,52.988247
1,Figueroa High School,District,2949.0,1884411.0,639.0,76.711767,81.158020,65.988471,80.739234,53.204476
2,Huang High School,District,2917.0,1910635.0,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
3,Hernandez High School,District,4635.0,3022020.0,652.0,77.289752,80.934412,66.752967,80.862999,53.527508
4,Johnson High School,District,4761.0,3094650.0,650.0,77.072464,80.966394,66.057551,81.222432,53.539172


In [19]:
# Group all students by school and grade using froupby
by_grade_grouped = school_data_complete.groupby(["school_name", "grade"])

In [20]:
# Math Scores by Grade
math_by_grade = by_grade_grouped["math_score"].mean()
math_by_grade

school_name            grade
Bailey High School     10th     76.996772
                       11th     77.515588
                       12th     76.492218
                       9th      77.083676
Cabrera High School    10th     83.154506
                       11th     82.765560
                       12th     83.277487
                       9th      83.094697
Figueroa High School   10th     76.539974
                       11th     76.884344
                       12th     77.151369
                       9th      76.403037
Ford High School       10th     77.672316
                       11th     76.918058
                       12th     76.179963
                       9th      77.361345
Griffin High School    10th     84.229064
                       11th     83.842105
                       12th     83.356164
                       9th      82.044010
Hernandez High School  10th     77.337408
                       11th     77.136029
                       12th     77.186567
     

In [21]:
# Reading Scores by Grade
reading_by_grade = by_grade_grouped["reading_score"].mean()
reading_by_grade

school_name            grade
Bailey High School     10th     80.907183
                       11th     80.945643
                       12th     80.912451
                       9th      81.303155
Cabrera High School    10th     84.253219
                       11th     83.788382
                       12th     84.287958
                       9th      83.676136
Figueroa High School   10th     81.408912
                       11th     80.640339
                       12th     81.384863
                       9th      81.198598
Ford High School       10th     81.262712
                       11th     80.403642
                       12th     80.662338
                       9th      80.632653
Griffin High School    10th     83.706897
                       11th     84.288089
                       12th     84.013699
                       9th      83.369193
Hernandez High School  10th     80.660147
                       11th     81.396140
                       12th     80.857143
     

In [22]:
# Scores by School Spending

# Initialize bins for grouping by size of school
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

# Cut the per_school_summary and convert to dataframe
spending_df = pd.cut(per_school_summary["Per Student Budget"], spending_bins, labels=labels, include_lowest=True)
spending_df = pd.DataFrame(spending_df)

#school_spending_df["School Name"] = per_school_summary["School Name"]
#school_spending_df.set_index("School Name", drop=True)
per_school_summary["Spending Ranges (Per Student)"] = spending_df["Per Student Budget"]

# Calculate each individual stat for the Spending Ranges
spending_math_scores = per_school_summary.groupby(["Spending Ranges (Per Student)"])["Average Math Score"].mean()
spending_reading_scores = per_school_summary.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
spending_passing_math = per_school_summary.groupby(["Spending Ranges (Per Student)"])["% Passing Math"].mean()
spending_passing_reading = per_school_summary.groupby(["Spending Ranges (Per Student)"])["% Passing Reading"].mean()
overall_passing_spending = per_school_summary.groupby(["Spending Ranges (Per Student)"])["% Overall Passing"].mean()

# Create a dataframe to add the stats to
spending_summary = pd.DataFrame(spending_math_scores)

# Add the remaining stats to the dataframe
for stat in [spending_reading_scores, spending_passing_math, spending_passing_reading, overall_passing_spending]:
    spending_summary = spending_summary.merge(stat, left_on='Spending Ranges (Per Student)', right_on='Spending Ranges (Per Student)')

spending_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,90.369459
$585-630,81.899826,83.155286,87.133538,92.718205,81.418596
$630-645,78.518855,81.624473,73.484209,84.391793,62.857656
$645-680,76.997210,81.027843,66.164813,81.133951,53.526855


In [23]:
# Scores by School Size

# Initialize bins for grouping by size of school
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

# Cut the per_school_summary and convert to dataframe
size_summary = pd.cut(per_school_summary["Total Students"], size_bins, labels=labels, include_lowest=True)
size_summary = pd.DataFrame(size_summary)

# Add categories to per_school_summary
per_school_summary["School Size Category"] = size_summary["Total Students"]

# Calculate each individual stat for the School Size
size_math_scores = per_school_summary.groupby(["School Size Category"])["Average Math Score"].mean()
size_reading_scores = per_school_summary.groupby(["School Size Category"])["Average Reading Score"].mean()
size_passing_math = per_school_summary.groupby(["School Size Category"])["% Passing Math"].mean()
size_passing_reading = per_school_summary.groupby(["School Size Category"])["% Passing Reading"].mean()
overall_passing_size = per_school_summary.groupby(["School Size Category"])["% Overall Passing"].mean()

# Create a dataframe to add the stats to
size_summary = pd.DataFrame(size_math_scores)

# Add the remaining stats to the dataframe
for stat in [size_reading_scores, size_passing_math, size_passing_reading, overall_passing_size]:
    size_summary = size_summary.merge(stat, left_on='School Size Category', right_on='School Size Category')

size_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size Category,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


In [24]:
# Scores by School Type

# Calculate each individual stat for the School Type
type_math_scores = per_school_summary.groupby(["School Type"])["Average Math Score"].mean()
type_reading_scores = per_school_summary.groupby(["School Type"])["Average Reading Score"].mean()
type_passing_math = per_school_summary.groupby(["School Type"])["% Passing Math"].mean()
type_passing_reading = per_school_summary.groupby(["School Type"])["% Passing Reading"].mean()
overall_passing_type = per_school_summary.groupby(["School Type"])["% Overall Passing"].mean()

# Create a dataframe to add the stats to
type_summary = pd.DataFrame(type_math_scores)

# Add the remaining stats to the dataframe
for stat in [type_reading_scores, type_passing_math, type_passing_reading, overall_passing_type]:
    type_summary = type_summary.merge(stat, left_on='School Type', right_on='School Type')

type_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208


In [33]:
per_school_summary.set_index("School Name", drop=True)
test = per_school_summary.groupby(["School Size Category", "School Type", "Spending Ranges (Per Student)"])

In [34]:
print(test)

In [35]:
for key, item in test:
    print(test.get_group(key), "\n\n")

          School Name School Type  Total Students  Total Budget  \
8  Holden High School     Charter           427.0      248087.0   

   Per Student Budget  Average Math Score  Average Reading Score  \
8               581.0           83.803279              83.814988   

   % Passing Math  % Passing Reading  % Overall Passing  \
8       92.505855          96.252927          89.227166   

  Spending Ranges (Per Student) School Size Category  
8                         <$585        Small (<1000)   


        School Name School Type  Total Students  Total Budget  \
9  Pena High School     Charter           962.0      585858.0   

   Per Student Budget  Average Math Score  Average Reading Score  \
9               609.0           83.839917              84.044699   

   % Passing Math  % Passing Reading  % Overall Passing  \
9       94.594595          95.945946          90.540541   

  Spending Ranges (Per Student) School Size Category  
9                      $585-630        Small (<1000)  